# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [40]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [41]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,acarau,-2.8856,-40.1200,26.79,79,86,4.66,BR,1704504645
1,1,puerto baquerizo moreno,-0.9000,-89.6000,24.63,82,89,4.53,EC,1704504677
2,2,wajir,1.7471,40.0573,25.07,68,9,4.94,KE,1704504677
3,3,ilulissat,69.2167,-51.1000,-7.99,45,100,13.38,GL,1704504678
4,4,grytviken,-54.2811,-36.5092,5.20,73,97,0.36,GS,1704504678
...,...,...,...,...,...,...,...,...,...,...
555,555,antalaha,-14.9003,50.2788,25.02,91,67,1.62,MG,1704504825
556,556,dezhou,37.4486,116.2925,-0.64,62,2,4.17,CN,1704504826
557,557,nyurba,63.2842,118.3319,-42.51,100,63,1.14,RU,1704504826
558,558,puerto madero,14.7208,-92.4215,25.11,100,0,1.03,MX,1704504826


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [44]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x='Lng', #Obtain x and y values for scatter plot from relevant Dataframe columns (Lng & Lat)
    y='Lat',
    c='Humidity', #Set colour to be variable based on Humidity column of Dataframe
    colorbar=True, #Add colour bar for heat map readability
    size='Humidity', #Size of scatter plot points set to be variable based on Humidity column of Dataframe
    title='City Map with Humidity', #Titles and labels setting
    xlabel='Longitude',
    ylabel='Latitude',
    hover_cols=['City', 'Humidity'], #Set plot to display City and Humidity values for that point when mouse hovers over
).opts(cmap='viridis', width=800, height=500) #Set colour mapping and specify dimensions of plot to make it more readable

# Display the map
map_plot

:Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[
    (city_data_df['Max Temp'] < 25) &
    (city_data_df['Humidity'] < 70) &
    (city_data_df['Wind Speed'] < 10) &
    (city_data_df['Cloudiness'] < 30)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,taoudenni,22.6783,-3.9836,18.41,19,2,0.38,ML,1704504678
14,14,edinburgh of the seven seas,-37.0676,-12.3116,14.36,67,18,5.96,SH,1704504680
20,20,galiakot,23.5167,74.0000,12.46,57,5,4.08,IN,1704504682
28,28,happy valley-goose bay,53.3168,-60.3315,-17.83,65,20,6.17,CA,1704504685
32,32,blackmans bay,-43.0167,147.3167,24.18,60,11,1.79,AU,1704504685
...,...,...,...,...,...,...,...,...,...,...
530,530,tindouf,27.6711,-8.1474,12.25,66,0,5.14,DZ,1704504821
537,537,fayetteville,35.0527,-78.8784,4.38,69,0,1.54,US,1704504822
541,541,beaufort west,-32.3567,22.5830,21.79,26,0,1.51,ZA,1704504823
552,552,lobujya,27.9500,86.8167,-18.12,58,0,1.26,NP,1704504825


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df = hotel_df.assign(Hotel_Name="")

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel_Name
6,6,taoudenni,22.6783,-3.9836,18.41,19,2,0.38,ML,1704504678,
14,14,edinburgh of the seven seas,-37.0676,-12.3116,14.36,67,18,5.96,SH,1704504680,
20,20,galiakot,23.5167,74.0000,12.46,57,5,4.08,IN,1704504682,
28,28,happy valley-goose bay,53.3168,-60.3315,-17.83,65,20,6.17,CA,1704504685,
32,32,blackmans bay,-43.0167,147.3167,24.18,60,11,1.79,AU,1704504685,
...,...,...,...,...,...,...,...,...,...,...,...
530,530,tindouf,27.6711,-8.1474,12.25,66,0,5.14,DZ,1704504821,
537,537,fayetteville,35.0527,-78.8784,4.38,69,0,1.54,US,1704504822,
541,541,beaufort west,-32.3567,22.5830,21.79,26,0,1.51,ZA,1704504823,
552,552,lobujya,27.9500,86.8167,-18.12,58,0,1.26,NP,1704504825,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [34]:
# Set parameters to search for a hotel
radius = 10000
params = { #Note that this is the base parameter dictionary for every search
    'limit': 1,  # Limit to one result (the nearest hotel)
    'apiKey': geoapify_key #Obtain API key from api_keys
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["categories"] = "accommodation.hotel"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
taoudenni - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
galiakot - nearest hotel: No hotel found
happy valley-goose bay - nearest hotel: Hotel North Two
blackmans bay - nearest hotel: Kingston Hotel
papatowai - nearest hotel: No hotel found
giado - nearest hotel: فندق جادو السياحي
altai - nearest hotel: Тайгам тал зочид буудал
bayanhongor - nearest hotel: No hotel found
tura - nearest hotel: No hotel found
iranshahr - nearest hotel: هتل قصِر
margaret river - nearest hotel: Quality Inn Margaret River
'alaqahdari dishu - nearest hotel: No hotel found
huazhou - nearest hotel: No hotel found
illapel - nearest hotel: Hotel Diaguitas
rio gallegos - nearest hotel: Avellaneda Hotel
invercargill - nearest hotel: The Grand
illizi - nearest hotel: فندق الطيب العربي مولاي
santa ana - nearest hotel: Metropolis
lajere - nearest hotel: No hotel found
ganye - nearest hotel: No hotel found
kijang - nearest hotel: 힐튼호텔 부산
tamanrasset - 

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel_Name,Hotel Name
6,6,taoudenni,22.6783,-3.9836,18.41,19,2,0.38,ML,1704504678,,No hotel found
14,14,edinburgh of the seven seas,-37.0676,-12.3116,14.36,67,18,5.96,SH,1704504680,,No hotel found
20,20,galiakot,23.5167,74.0000,12.46,57,5,4.08,IN,1704504682,,No hotel found
28,28,happy valley-goose bay,53.3168,-60.3315,-17.83,65,20,6.17,CA,1704504685,,Hotel North Two
32,32,blackmans bay,-43.0167,147.3167,24.18,60,11,1.79,AU,1704504685,,Kingston Hotel
...,...,...,...,...,...,...,...,...,...,...,...,...
530,530,tindouf,27.6711,-8.1474,12.25,66,0,5.14,DZ,1704504821,,محمد بوسبي
537,537,fayetteville,35.0527,-78.8784,4.38,69,0,1.54,US,1704504822,,"Home2 Suites by Hilton Fayetteville, NC"
541,541,beaufort west,-32.3567,22.5830,21.79,26,0,1.51,ZA,1704504823,,Wagon Wheel
552,552,lobujya,27.9500,86.8167,-18.12,58,0,1.26,NP,1704504825,,Himalaya Lodge


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [35]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    x='Lng', #Obtain x and y values for scatter plot from relevant Dataframe columns (Lng & Lat)
    y='Lat',
    c='Humidity', #Set colour to be variable based on Humidity column of Dataframe
    colorbar=True, #Add colour bar for heat map readability
    size='Humidity', #Size of scatter plot points set to be variable based on Humidity column of Dataframe
    title='City Map with Humidity', #Titles and labels setting
    xlabel='Longitude',
    ylabel='Latitude',
    hover_cols=['City', 'Humidity', 'Hotel Name', 'Country'] #Set plot to display City, Humidity, Hotel Name and Country values for that point when mouse hovers over
).opts(cmap='viridis', width=800, height=500) #Set colour mapping and specify dimensions of plot to make it more readable

# Display the map
map_plot

:Points   [Lng,Lat]   (Humidity,City,Hotel Name,Country)